In [7]:
#%pip install nltk
#%pip install pandas
#%pip install numpy
#%pip install scikit-learn
#%pip install spacy
#%pip install pymorphy2
#%pip install gensim

In [8]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import spacy
import sklearn
import pymorphy2
from pymorphy2 import MorphAnalyzer
import string
import re
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
import gensim

In [9]:
#define
nltk.download('stopwords')
nltk.download('punkt')
rus = "russian"
sw = stopwords.words(rus)
morph = MorphAnalyzer()
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
lancaster = LancasterStemmer()
snowball = SnowballStemmer(language=rus)
vectorizer = CountVectorizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NitghtWay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NitghtWay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
test_ds1_path = "C:\\course_work\\feature_extraction_NLP\\data\\test_ds1\\train.jsonl"

In [11]:
data = pd.read_json(test_ds1_path, lines=True)
data

,premise,label,hypothesis,verb,negation,genre,idx,no_negation
0,Сумма ущерба составила одну тысячу рублей. Уто...,entailment,Ранее местный житель совершал подобное правона...,судить,no_negation,kp,0,NaN
1,"Перебрасываясь словечками, они скользят глазам...",contradiction,Они что-то понимают,смотреть,no_negation,fiction,1,NaN
2,"— Разве что, — сказала она, — мы хотим где-ниб...",neutral,"Это “ Таганская ”, а не “ Тульская ”.",понять,no_negation,fiction,2,NaN
3,"Зима, наконец, показала свой характер.",contradiction,У зимы есть свой характер.,показать,no_negation,kp,3,NaN
4,ГуманностьБогаподобнымсценариемнепредполагаетс...,contradiction,Это должно быть просто.,сказать,no_negation,fiction,4,NaN
...,...,...,...,...,...,...,...,...
433,Тверские спортсмены показали отличный результа...,entailment,У тверских спортсменов пять новых медалей.,суметь,no_negation,kp,433,NaN
434,"Просто я не хочу с ним жить и, извини, больше ...",contradiction,Этого было вполне достаточно.,уверять,no_negation,fiction,434,NaN
435,"Второй аргумент: все читаем и видим, что надви...",neutral,Надвигается вторая волна кризиса.,видеть,negation,interfax,435,NaN
436,"— Да, сын. Здравствуй. — Ты можешь говорить, н...",neutral,Сын не занят,говорить,no_negation,fiction,436,NaN


In [12]:
type(data)

pandas.core.frame.DataFrame

In [13]:
data = data.drop(['label', 'hypothesis', 'verb','negation','genre','idx','no_negation'], axis=1)

In [14]:
data.head()

,premise
0,Сумма ущерба составила одну тысячу рублей. Уто...
1,"Перебрасываясь словечками, они скользят глазам..."
2,"— Разве что, — сказала она, — мы хотим где-ниб..."
3,"Зима, наконец, показала свой характер."
4,ГуманностьБогаподобнымсценариемнепредполагаетс...


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   premise  438 non-null    object
dtypes: object(1)
memory usage: 3.5+ KB


In [16]:
data.shape

(438, 1)

Вспомогательные методы

In [17]:
def print_table(array):
    table_width = 5
    for i in range(0, len(array), table_width):
        row = array[i:i + table_width]
        formatted_row = " ".join(["{:<10}".format(str(elem)) for elem in row])
        print(formatted_row)

In [18]:
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in 
    string.punctuation])

In [19]:
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text]) 
    # метод для удаления чисел

In [20]:
def remove_patterns(text):
    text = re.sub(patterns, ' ', text)
    return text

In [21]:
def remove_notalpha(text):
    return ''.join(i if i.isalpha() else ' ' for i in text) # 

Стемминг с использованием SnowBall

In [22]:
example_text = data['premise'][4]
print(type(example_text))
print(example_text)

<class 'str'>
ГуманностьБогаподобнымсценариемнепредполагается. Но Его благость остается в неприкосновенности. Непросто жить в таком мире, но кто сказал, что это должно быть просто?


In [23]:
def tokenize_snowball(text):
    text = remove_notalpha(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = remove_patterns(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw or len(w) >= 11:
                continue
            else:
                all_words.append(snowball.stem(w))
            
    return all_words

In [24]:
print_table(tokenize_snowball(example_text))

но         ег         благост    оста       непрост   
жит        так        мир        сказа      эт        
должн      прост     


In [25]:
lancaster = LancasterStemmer()
def tokenize_lancaster(text):
    text = remove_notalpha(text)
    text = remove_numbers(text)
    text = remove_patterns(text)
    text = remove_punctuation(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw or len(w) >= 11:
                continue
            else:
                all_words.append(lancaster.stem(w))
    return all_words

In [26]:
print_table(tokenize_lancaster(example_text))

но         его        благость   остается   непросто  
жить       таком      мире       сказал     это       
должно     просто    


Лемматизация с помощью PyMorphy2

In [27]:
def tokenize_morphy(text):
    text = remove_notalpha(text)
    text = remove_patterns(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw or len(w) >= 11:
                continue
            else:
                all_words.append(morph.normal_forms(w)[0])
    if len(all_words) > 2:
        #all_words = pd.Series(all_words)
        return all_words
    else:
        return None
    

In [28]:
print_table(tokenize_morphy(example_text))

но         он         благость   оставаться непросто  
жить       такой      мир        сказать    это       
должный    просто    


In [29]:
df_morphy = pd.DataFrame({'text':[tokenize_morphy(sample) for sample in data['premise']]})
df_snowball = pd.DataFrame({'text':[tokenize_snowball(sample) for sample in data['premise']]})
df_lancaster = pd.DataFrame({'text':[tokenize_lancaster(sample) for sample in data['premise']]})
df_lancaster = df_lancaster.dropna()
df_morphy = df_morphy.dropna()
df_snowball = df_snowball.dropna()

In [30]:
df_morphy.head()

,text
0,"[сумма, ущерб, составить, один, тысяча, рубль,..."
1,"[словечко, скользить, глаз, мой, город, как, с..."
2,"[разве, сказать, хотеть, выпить, кофе, я, каза..."
3,"[зима, показать, свой, характер]"
4,"[но, он, благость, оставаться, непросто, жить,..."


In [31]:
df_snowball.head()

,text
0,"[сумм, ущерб, состав, одн, тысяч, рубл, уточня..."
1,"[словечк, скольз, глаз, мо, город, как, смеют,..."
2,"[разв, сказа, хот, вып, коф, мне, кажет, сказа..."
3,"[зим, показа, сво, характер]"
4,"[но, ег, благост, оста, непрост, жит, так, мир..."


In [32]:
df_lancaster.head()

,text
0,"[сумма, ущерба, составила, одну, тысячу, рубле..."
1,"[словечками, скользят, глазами, моему, городу,..."
2,"[разве, сказала, хотим, выпить, кофе, мне, каж..."
3,"[зима, показала, свой, характер]"
4,"[но, его, благость, остается, непросто, жить, ..."


Векторизация текста с использованием словаря

In [33]:
def vectorize(doc):
    features = defaultdict(int)
    for token in doc:
        features[token] += 1
    return features

In [34]:
def make_dictionary_words(df):
    dictionary_whole_words = defaultdict(int)
    for doc in df['text']:
        for token in doc:
            dictionary_whole_words[token] += 1
    return dictionary_whole_words

In [35]:
def sentence_to_vector(sentence, dictionary_whole_words):
    sentence = np.array(sentence)
    vector = np.zeros(len(dictionary_whole_words))
    dict_values = np.array(list(dictionary_whole_words.values()))
    dict_keys = np.array(list(dictionary_whole_words.keys()))
    sorter = np.argsort(dict_keys)
    for idx in sorter[np.searchsorted(dict_keys, sentence, sorter = sorter)]:
        vector[idx] = dict_values[idx]
    return vector 

In [36]:
whole_dict = make_dictionary_words(df_morphy)
whole_dict

defaultdict(int,
            {'сумма': 4,
             'ущерб': 2,
             'составить': 3,
             'один': 22,
             'тысяча': 4,
             'рубль': 4,
             'уточняться': 3,
             'место': 16,
             'выехать': 3,
             'группа': 5,
             'который': 34,
             'установить': 8,
             'личность': 2,
             'они': 14,
             'оказаться': 14,
             'местный': 4,
             'житель': 7,
             'ранее': 7,
             'судить': 9,
             'подобный': 3,
             'словечко': 2,
             'скользить': 2,
             'глаз': 8,
             'мой': 25,
             'город': 6,
             'как': 26,
             'сметь': 5,
             'смотреть': 8,
             'понимать': 40,
             'разве': 3,
             'сказать': 122,
             'хотеть': 28,
             'выпить': 2,
             'кофе': 1,
             'я': 81,
             'казаться': 13,
             'на': 8,
       

In [37]:
print(sentence_to_vector(df_morphy['text'][1], whole_dict)[20:])

[2. 2. 8. ... 0. 0. 0.]


In [38]:
a = defaultdict(int)
a["aboba"] += 1
a["g"] += 1
a["ggg"] += 1
B = np.array(list(a.keys()))
print(B)

['aboba' 'g' 'ggg']


In [39]:
for doc in df_snowball['text']:
    print(vectorize(doc))

defaultdict(<class 'int'>, {'сумм': 1, 'ущерб': 1, 'состав': 1, 'одн': 1, 'тысяч': 1, 'рубл': 1, 'уточня': 1, 'мест': 1, 'выеха': 1, 'групп': 1, 'котор': 1, 'установ': 1, 'личност': 1, 'им': 1, 'оказа': 1, 'местн': 1, 'жител': 1, 'ран': 1, 'судим': 1, 'подобн': 1})
defaultdict(<class 'int'>, {'словечк': 1, 'скольз': 1, 'глаз': 1, 'мо': 1, 'город': 1, 'как': 1, 'смеют': 1, 'смотрет': 1, 'понима': 1})
defaultdict(<class 'int'>, {'разв': 1, 'сказа': 2, 'хот': 2, 'вып': 1, 'коф': 1, 'мне': 1, 'кажет': 1, 'на': 1, 'след': 1, 'он': 1, 'успел': 1, 'встат': 1, 'эскалатор': 1, 'поня': 1, 'эт': 1, 'таганск': 1, 'тульск': 1})
defaultdict(<class 'int'>, {'зим': 1, 'показа': 1, 'сво': 1, 'характер': 1})
defaultdict(<class 'int'>, {'но': 1, 'ег': 1, 'благост': 1, 'оста': 1, 'непрост': 1, 'жит': 1, 'так': 1, 'мир': 1, 'сказа': 1, 'эт': 1, 'должн': 1, 'прост': 1})
defaultdict(<class 'int'>, {'для': 1, 'мам': 1, 'праздник': 1, 'обычн': 1, 'ден': 1, 'он': 1, 'корм': 1, 'люд': 1, 'весен': 1, 'неб': 1, 'о

In [40]:
for doc in df_lancaster['text']:
    print(vectorize(doc))

defaultdict(<class 'int'>, {'сумма': 1, 'ущерба': 1, 'составила': 1, 'одну': 1, 'тысячу': 1, 'рублей': 1, 'уточняется': 1, 'место': 1, 'выехала': 1, 'группа': 1, 'которая': 1, 'установила': 1, 'личность': 1, 'им': 1, 'оказался': 1, 'местный': 1, 'житель': 1, 'ранее': 1, 'судимый': 1, 'подобное': 1})
defaultdict(<class 'int'>, {'словечками': 1, 'скользят': 1, 'глазами': 1, 'моему': 1, 'городу': 1, 'как': 1, 'смеют': 1, 'смотреть': 1, 'понимают': 1})
defaultdict(<class 'int'>, {'разве': 1, 'сказала': 1, 'хотим': 2, 'выпить': 1, 'кофе': 1, 'мне': 1, 'кажется': 1, 'сказал': 1, 'на': 1, 'следующей': 1, 'он': 1, 'успел': 1, 'встать': 1, 'эскалатор': 1, 'понял': 1, 'это': 1, 'таганская': 1, 'тульская': 1})
defaultdict(<class 'int'>, {'зима': 1, 'показала': 1, 'свой': 1, 'характер': 1})
defaultdict(<class 'int'>, {'но': 1, 'его': 1, 'благость': 1, 'остается': 1, 'непросто': 1, 'жить': 1, 'таком': 1, 'мире': 1, 'сказал': 1, 'это': 1, 'должно': 1, 'просто': 1})
defaultdict(<class 'int'>, {'для':

Векторизация текста sklearn

In [41]:
corpus = data['premise']

In [42]:
vectors = vectorizer.fit_transform(corpus)
print(vectors)

  (0, 3932)	1
  (0, 4328)	1
  (0, 3759)	1
  (0, 2220)	1
  (0, 4129)	1
  (0, 3411)	1
  (0, 4296)	1
  (0, 4478)	1
  (0, 1775)	1
  (0, 1670)	1
  (0, 3118)	1
  (0, 489)	1
  (0, 3601)	1
  (0, 670)	1
  (0, 1443)	1
  (0, 4279)	1
  (0, 1553)	1
  (0, 1147)	1
  (0, 1222)	1
  (0, 2230)	1
  (0, 1669)	1
  (0, 969)	1
  (0, 3282)	1
  (0, 3916)	1
  (0, 978)	1
  :	:
  (435, 292)	1
  (435, 1795)	1
  (435, 1479)	1
  (435, 2418)	1
  (435, 1379)	1
  (435, 3264)	1
  (436, 1925)	1
  (436, 677)	1
  (436, 4126)	1
  (436, 590)	1
  (436, 3972)	1
  (436, 1072)	1
  (436, 1138)	1
  (436, 1720)	1
  (437, 4478)	1
  (437, 2537)	1
  (437, 3564)	1
  (437, 3514)	1
  (437, 3999)	1
  (437, 1663)	1
  (437, 3010)	1
  (437, 3820)	1
  (437, 4017)	1
  (437, 4230)	1
  (437, 37)	1


Векторизация текста Gensim

In [43]:
corpus = [doc for doc in df_morphy['text']]
id2word = gensim.corpora.Dictionary(corpus)
vectors = [id2word.doc2bow(doc) for doc in corpus]
for v in vectors:
    print(v)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]
[(20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]
[(29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1)]
[(46, 1), (47, 1), (48, 1), (49, 1)]
[(34, 1), (37, 1), (44, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)]
[(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1)]
[(13, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1)]
[(11, 1), (34, 1), (72, 1), (73, 1), (80, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1)]
[(8, 1), (40, 1), (41, 1), (43, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98

Реализация Bag of words

In [44]:
text_series = df_morphy['text']
allsentences = []
for sent in text_series:
    allsentences.append(' '.join(sent))
allsentences = pd.Series(allsentences)
print(allsentences)

0      сумма ущерб составить один тысяча рубль уточня...
1      словечко скользить глаз мой город как сметь см...
2      разве сказать хотеть выпить кофе я казаться ск...
3                            зима показать свой характер
4      но он благость оставаться непросто жить такой ...
                             ...                        
432    тверской спортсмен показать отличный результат...
433    просто хотеть жить извинить сказать ты нечего ...
434    второй аргумент читаемый видеть второй волна к...
435           да сын здравствуй ты мочь говорить занятый
436        там сказать мера спуск тело умерший альпинист
Length: 437, dtype: object


In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(allsentences)
print(bow_matrix.toarray()) 
#print(get_similarity_values(bow_matrix))
evaluate(get_similarity_values, "bbb")

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


NameError: name 'evaluate' is not defined

In [ ]:
feature_names = vectorizer.get_feature_names_out()
print(len(feature_names))
print(df_morphy.info())
print(len(df_morphy['text']))
feature_names

2532
<class 'pandas.core.frame.DataFrame'>
Index: 437 entries, 0 to 437
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    437 non-null    object
dtypes: object(1)
memory usage: 23.0+ KB
None
437


array(['аварийный', 'авария', 'авиасалон', ..., 'ярость', 'ясно',
       'ясность'], dtype=object)